In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt
import json
from datetime import datetime, timedelta

In [12]:
driver = webdriver.Chrome("C:/Users/김근영/chromedriver_win32.zip/chromedriver.exe")

C:\Users\김근영\AppData\Local\Temp\ipykernel_7212\3107180718.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/김근영/chromedriver_win32.zip/chromedriver.exe")


In [13]:
sm_genre_dict = { 
    3 : "가정/생활", #요리/살림
    4 : "가정/생활", #건강/취미
    5 : "경제/비즈니스", #경제/경영
#    6 : "문학", #고전
    7 : "자연/과학", #과학, 사회과
    8 : "강의지원도서", #대학교재/전문서적
#    9 : "라이트 노벨",
    10 : "문학", #소설/시/희곡
#    11 : "사전/기타",
    12 : "인문", #사회과학
#    13 : "수험서/자격증",
#    14 : "어린이",
    15 : "에세이/산문", # 에세이, 자기계발
#    16 : "여행",
#    17 : "역사",
    18 : "문화/예술", # 예술/대중문화
    19 : "외국어", #외국어
#    20 : "유아",
    21 : "인문", #인문학
#    22 : "인물/평전",
    23 : "에세이/산문",
#    24 : "잡지",
#    25 : "전집/중고전집",
#    26 : "종교/역학",
    27 : "가정/생활", #좋은부모
#    28 : "참고서/학습서",
#    29 : "청소년",
    30 : "컴퓨터/인터넷", # 컴퓨터/모바일
}

In [14]:

aladin_genre_dict = { 
    3 : "요리/살림",
    4 : "건강/취미",
    5 : "경제경영",
    # 6 :/ "고전",
    7 : "과학",
    8 : "대학교재/전문서적",
    # 9 : "라이트 노벨",
    10 : "소설/시/희곡",
    # 11 : "사전/기타",
    12 : "사회과학",
    # 13 : "수험서/자격증",
    # 14 : "어린이",
    15 : "에세이",
    # 16 : "여행",
    # 17 : "역사",
    18 : "예술/대중문화",
    19 : "외국어",
    # 20 : "유아",
    21 : "인문학",
    # 22 : "인물/평전",
    23 : "자기계발",
    # 24 : "잡지",
    # 25 : "전집/중고전집",
    # 26 : "종교/역학",
    27 : "좋은 부모",
    # 28 : "참고서/학습서",
    # 29 : "청소년",
    30 : "컴퓨터/모바일"
}

In [15]:
_index = "aladin_best_100_"

In [16]:
# 칼럼 리스트 준비
book_list = []

In [17]:
present_date = str(datetime.utcnow() + timedelta(hours = 9))[:10]

In [18]:
driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=38409")

In [19]:
for sm_genre_num in range(3, 30):
    
    if(sm_genre_num == 3):
        sm_genre_id = 38409
        
    if(sm_genre_num == 6 or sm_genre_num == 9 or sm_genre_num == 11 or sm_genre_num == 13 or sm_genre_num == 14 or sm_genre_num == 16 or sm_genre_num == 17
       or sm_genre_num == 20 or sm_genre_num == 22 or sm_genre_num == 24 or sm_genre_num == 25 or sm_genre_num == 26 or sm_genre_num == 28
       or sm_genre_num == 29):
    # if(sm_genre_num == 6 or 9 or 11 or 13 or 14 or 16 or 17 or 20 or 22 or 24 or 25 or 28 or 29):
        sm_genre_num += 1
        continue

    elif(sm_genre_num == 4):
        sm_genre_id = 56388
    elif(sm_genre_num == 5):
        sm_genre_id = 38398
    elif(sm_genre_num == 6):
        sm_genre_id = 38414
    elif(sm_genre_num == 7):
        sm_genre_id = 38405
    elif(sm_genre_num == 8):
        sm_genre_id = 38422
   
    elif(sm_genre_num == 10):
        sm_genre_id = 38396

    elif(sm_genre_num == 12):
        sm_genre_id = 38404

    elif(sm_genre_num == 15):
        sm_genre_id = 56387

    elif(sm_genre_num == 18):
        sm_genre_id = 38402
    elif(sm_genre_num == 19):
        sm_genre_id = 38411

    elif(sm_genre_num == 21):
        sm_genre_id = 38403

    elif(sm_genre_num == 23):
        sm_genre_id = 38400

    elif(sm_genre_num == 27):
        sm_genre_id = 38413

        
    sm_genre_num += 1
    time.sleep(1)

    for i in range(1, 3):
        driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID="+str(sm_genre_id)+"&page="+str(i)+"&cnt=300&SortOrder=1")
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 1위 책 클릭(대여 횟수 긁어오기 위함)

        books = soup.select('div.ss_book_box')
        def get_star(book):
            stars = book.select_one("div.ss_book_list img")
            if stars is None:
                return 0
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s10.gif":
                return 5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s9.gif":
                return 4.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s8.gif":
                return 4
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s7.gif":
                return 3.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s6.gif":
                return 3
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s5.gif":
                return 2.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s4.gif":
                return 2
            else:
                return 0
        
        for book in books:
            try: #일반적인 경우
                title = book.select('a.bo3')[0].text
                rank = book.select("td")[0].text
                
                li_tags = book.find_all('li')
                second_li_tag = li_tags[2] # 2. 두 번째 li 태그 선택
                info = second_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                author=info.split('|')[0]
                publisher=info.split('|')[1]
                date=info.split('|')[2] 
                
                price = book.select("span.ss_p2 > b > span")[0].text
                sm_genre = sm_genre_dict[sm_genre_num - 1]
                aladin_genre = aladin_genre_dict[sm_genre_num - 1]
                
                star = get_star(book)
                
                i += 1
                

            except IndexError: #행사 상품
                title = book.select('a.bo3')[0].text
                price = book.select("span.ss_p2 > b > span")[0].text
                rank = book.select("td")[0].text
                
                star = get_star(book)
                try: #only 행사 상품
                    li_tags = book.find_all('li')
                    third_li_tag = li_tags[3] # 2. 두 번째 li 태그 선택
                    info = third_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                    author=info.split('|')[0]
                    publisher=info.split('|')[1]
                    date=info.split('|')[2]
                    sm_genre = sm_genre_dict[sm_genre_num - 1]
                    aladin_genre = aladin_genre_dict[sm_genre_num - 1]
                    

                except IndexError: #행사 상품은 아닌데, 출판사가 없음.
                    second_li_tag = li_tags[2]
                    info = second_li_tag.get_text()
                    author=info.split('|')[0]
                    publisher=None
                    date=info.split('|')[1] 
                    sm_genre = sm_genre_dict[sm_genre_num - 1]
                    aladin_genre = aladin_genre_dict[sm_genre_num - 1]

               
            book_list.append([rank, title, author, publisher, date, price, star, aladin_genre, sm_genre, present_date])
            i += 1

In [20]:
for i in range(1, 3):
    driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=38401&page="+str(i)+"&cnt=300&SortOrder=1")
    time.sleep(0.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 1위 책 클릭(대여 횟수 긁어오기 위함)
    books = soup.select('div.ss_book_box')
    def get_star(book):
        stars = book.select_one("div.ss_book_list img")
        if stars is None:
            return 0
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s10.gif":
            return 5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s9.gif":
            return 4.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s8.gif":
            return 4
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s7.gif":
            return 3.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s6.gif":
            return 3
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s5.gif":
            return 2.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s4.gif":
            return 2
        else:
            return 0
    for book in books:
        title = book.select('a.bo3')[0].text
        price = book.select("span.ss_p2 > b > span")[0].text
        rank = book.select("td")[0].text
                
        star = get_star(book)
        try: #일반적인 경우
            
            li_tags = book.find_all('li')
            second_li_tag = li_tags[2] # 2. 두 번째 li 태그 선택
            info = second_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

            author=info.split('|')[0]
            publisher=info.split('|')[1]
            date=info.split('|')[2] 
            price = book.select("span.ss_p2 > b > span")[0].text
            sm_genre = sm_genre_dict[sm_genre_num]            
                
            i += 1
            

        except IndexError: #행사 상품
            sm_genre = sm_genre_dict[sm_genre_num]
            aladin_genre = aladin_genre_dict[sm_genre_num]
            try: #only 행사 상품
                li_tags = book.find_all('li')
                third_li_tag = li_tags[3] # 2. 두 번째 li 태그 선택
                info = third_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                author=info.split('|')[0]
                publisher=info.split('|')[1]
                date=info.split('|')[2]
                
            
            except IndexError: #행사 상품은 아닌데, 출판사가 없음.
                second_li_tag = li_tags[2]
                info = second_li_tag.get_text()
                author=info.split('|')[0]
                publisher=None
                date=info.split('|')[1] 
                
            

        book_list.append([rank, title, author, publisher, date, price, star, aladin_genre, sm_genre, present_date])
        i += 1

In [21]:
df = pd.DataFrame(book_list, columns = ["aladin_rank", "subject", "aladin_writer", "aladin_publisher", "aladin_date","aladin_price", "aladin_review", "aladin_genre", "sm_genre", "present_date"])

df['present_date'] = present_date #time_stamp 속성 생성



In [22]:
df

,aladin_rank,subject,aladin_writer,aladin_publisher,aladin_date,aladin_price,aladin_review,aladin_genre,sm_genre,present_date
0,\n\n1. \n,"밥 챙겨 먹어요, 행복하세요",마포농수산쎈타 (지은이),세미콜론,2023년 1월,"13,800",5.0,요리/살림,가정/생활,2023-02-24
1,\n\n2. \n,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우 (지은이),빅피시,2022년 5월,"13,160",5.0,요리/살림,가정/생활,2023-02-24
2,\n\n3. \n,요리하는 다이어터의 맛있게 살 빼는 다이어트 레시피\t,이은경 (지은이),메가스터디BOOKS\t,2023년 1월,"12,600",5.0,요리/살림,가정/생활,2023-02-24
3,\n\n4. \n,시미켄의 베스트 섹스,"시미즈 켄 (지은이), 김봄 (옮긴이)",ㅁㅅㄴ,2018년 2월,"12,000",5.0,요리/살림,가정/생활,2023-02-24
4,\n\n5. \n,김대리의 쉽게 뜨는 요즘 니트,김대리(바늘이야기) (지은이),웅진리빙하우스,2022년 1월,"13,800",4.5,요리/살림,가정/생활,2023-02-24
...,...,...,...,...,...,...,...,...,...,...
1395,\n\n96. \n,이것이 C#이다,박상현 (지은이),한빛미디어,2021년 1월,"24,480",4.5,좋은 부모,컴퓨터/인터넷,2023-02-24
1396,\n\n97. \n,CODE 코드,"찰스 펫졸드 (지은이), 김현규 (옮긴이)",인사이트,2020년 8월,"20,000",4.5,좋은 부모,컴퓨터/인터넷,2023-02-24
1397,\n\n98. \n,레트로의 유니티 게임 프로그래밍 에센스,이제민 (지은이),한빛미디어,2022년 2월,"43,200",5.0,좋은 부모,컴퓨터/인터넷,2023-02-24
1398,\n\n99. \n,그림으로 공부하는 IT 인프라 구조,"야마자키 야스시, 미나와 요시코, 아제카츠 요헤이 (지은이)",제이펍,2021년 4월,"18,200",5.0,좋은 부모,컴퓨터/인터넷,2023-02-24


In [23]:

# 데이터 전처리

df['aladin_rank'] = df["aladin_rank"].str.replace(pat = r'[^A-Za-z0-9가-힣]', repl = r' ', regex = True) #공백(엔터)

df['aladin_price'] = df['aladin_price'].str.replace(pat=',', repl = '',regex=True) #, 삭제
df['aladin_price'] = pd.to_numeric(df['aladin_price'])


df['aladin_date'] = df['aladin_date'].str.replace(pat='년 ', repl = '-',regex=True)#년 -> -
df['aladin_date'] = df['aladin_date'].str.replace(pat='월', repl = '',regex=True) #월 -> 

df['aladin_date'] = df['aladin_date'].str.strip()
df['aladin_date'] = pd.to_datetime(df['aladin_date'])

In [24]:
df2 = df

In [25]:
from elasticsearch import Elasticsearch, helpers
import configparser

In [26]:
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [27]:
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

C:\Users\김근영\AppData\Local\Temp\ipykernel_7212\2962435662.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [28]:
if es.indices.exists(index = _index + present_date):
    pass
else:
    resp = es.indices.create(index = _index + present_date, body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori": {
                        "tokenizer": "nori_tokenizer"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "subject": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_genre": {
                    "type": "keyword"
                },
                "aladin_writer": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_publisher": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_date": {
                    "type": "text",
                    "fields" : {
                        "keyword" : {
                            "type" : "keyword"
                        }
                    }
                },
                "sm_genre": {
                    "type": "keyword"
                },
                "aladin_price": {
                    "type": "integer"
                },
                "aladin_review": {
                    "type": "float"
                },
                "present_date": {
                    "type": "date"
                }
            }
        }
    })

C:\Users\김근영\AppData\Local\Temp\ipykernel_7212\4125453572.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.indices.create(index = _index + present_date, body = {


In [29]:
for k in range(len(df2)) :
    es.index(index= _index + present_date,
         document = {
         "aladin_rank" : str(df2.loc[k].aladin_rank),
         "subject" : str(df2.loc[k].subject),
         "aladin_writer": str(df2.loc[k].aladin_writer),
         "aladin_publisher": str(df2.loc[k].aladin_publisher),
         "aladin_date": str(df2.loc[k].aladin_date),
         "aladin_price": str(df2.loc[k].aladin_price),
         "aladin_review": str(df2.loc[k].aladin_review),
         "aladin_genre": str(df2.loc[k].aladin_genre),
         "sm_genre": str(df2.loc[k].sm_genre),
         "present_date": present_date
 })